<a href="https://colab.research.google.com/github/arunaimlprojects/lora_fine_tuning/blob/main/lora_fine_tuning25.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi


Tue Mar 25 05:03:21 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   41C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
!pip install bitsandbytes transformers accelerate peft -q
!pip install git+https://github.com/huggingface/diffusers.git -q


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 57.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 47.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 54.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!mkdir -p /content/drive/MyDrive/woman_dataset


In [ ]:
from transformers import AutoProcessor, BlipForConditionalGeneration
import torch
from PIL import Image
import glob
import json

device = "cuda" if torch.cuda.is_available() else "cpu"

blip_processor = AutoProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
blip_model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base", torch_dtype=torch.float16).to(device)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/506 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.56k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

In [ ]:
def caption_images(image_path):
    image = Image.open(image_path).convert("RGB")
    inputs = blip_processor(images=image, return_tensors="pt").to(device, torch.float16)
    generated_ids = blip_model.generate(**inputs, max_length=50)
    caption = blip_processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
    return caption

image_folder = "/content/drive/MyDrive/woman_dataset"
image_paths = glob.glob(f"{image_folder}/*.jpg")  # Ensure images are in JPG format

metadata = []
for img_path in image_paths:
    caption = f"a photo of kavya women, {caption_images(img_path)}"
    metadata.append({"file_name": img_path.split("/")[-1], "prompt": caption})

# Save metadata
with open(f"{image_folder}/metadata.jsonl", "w") as f:
    for entry in metadata:
        json.dump(entry, f)
        f.write("\n")


In [ ]:
import gc
del blip_processor, blip_model
gc.collect()
torch.cuda.empty_cache()


In [ ]:
!accelerate config default


accelerate configuration saved at /root/.cache/huggingface/accelerate/default_config.yaml


In [ ]:
!wget https://raw.githubusercontent.com/huggingface/diffusers/main/examples/dreambooth/train_dreambooth_lora_sdxl.py


--2025-03-25 05:18:28--  https://raw.githubusercontent.com/huggingface/diffusers/main/examples/dreambooth/train_dreambooth_lora_sdxl.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84961 (83K) [text/plain]
Saving to: ‘train_dreambooth_lora_sdxl.py’

train_dreambooth_lo 100%[===================>]  82.97K  --.-KB/s    in 0.01s   

2025-03-25 05:18:29 (6.75 MB/s) - ‘train_dreambooth_lora_sdxl.py’ saved [84961/84961]



In [ ]:
import shutil

# Move metadata.jsonl to outside the dataset folder
shutil.move("/content/drive/MyDrive/woman_dataset/metadata.jsonl", "/content/drive/MyDrive/metadata.jsonl")

print("metadata.jsonl has been moved successfully!")


metadata.jsonl has been moved successfully!


In [ ]:
!accelerate launch train_dreambooth_lora_sdxl.py \
  --pretrained_model_name_or_path="stabilityai/stable-diffusion-xl-base-1.0" \
  --pretrained_vae_model_name_or_path="madebyollin/sdxl-vae-fp16-fix" \
  --instance_data_dir="/content/drive/MyDrive/woman_dataset" \
  --output_dir="/content/drive/MyDrive/woman_sdxl_LoRA" \
  --caption_column="prompt" \
  --mixed_precision="fp16" \
  --instance_prompt="a photo of kavya woman" \
  --resolution=1024 \
  --train_batch_size=1 \
  --gradient_accumulation_steps=3 \
  --gradient_checkpointing \
  --learning_rate=1e-4 \
  --snr_gamma=5.0 \
  --lr_scheduler="constant" \
  --lr_warmup_steps=0 \
  --use_8bit_adam \
  --max_train_steps=500 \
  --checkpointing_steps=717 \
  --seed="0"


2025-03-25 05:27:49.543011: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742880469.565881    8842 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742880469.573044    8842 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
03/25/2025 05:27:54 - INFO - __main__ - Distributed environment: DistributedType.NO
Num processes: 1
Process index: 0
Local process index: 0
Device: cuda

Mixed precision type: fp16

You are using a model of type clip_text_model to instantiate a model of type . This is not supported for all configurations of models and can yield errors.
You are using a model of type clip_text_model to instantiate a model of type . This is not support

In [ ]:
from diffusers import StableDiffusionXLPipeline
import torch

# Model path (जहां आपने LoRA trained model save किया है)
model_path = "/content/drive/MyDrive/woman_sdxl_LoRA/pytorch_lora_weights.safetensors"


In [ ]:
# Pipeline load करें
pipeline = StableDiffusionXLPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    torch_dtype=torch.float16
).to("cuda")

# LoRA weights लोड करें
pipeline.unet.load_attn_procs(model_path)


Fetching 19 files:   0%|          | 0/19 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/diffusers/loaders/unet.py:212: FutureWarning: `load_attn_procs` is deprecated and will be removed in version 0.40.0. Using the `load_attn_procs()` method has been deprecated and will be removed in a future version. Please use `load_lora_adapter()`.
  deprecate("load_attn_procs", "0.40.0", deprecation_message)


In [ ]:
# Prompt दें (जो image generate करनी है)
prompt = "A realistic portrait of a beautiful Indian woman, wearing a red saree, smiling, highly detailed, 4K resolution"

# Image generate करें
image = pipeline(prompt=prompt, num_inference_steps=50, guidance_scale=7.5).images[0]

# Image दिखाएं
image.show()

# Image save करें
image.save("generated_image.png")


  0%|          | 0/50 [00:00<?, ?it/s]

In [ ]:
# Prompt दें (जो image generate करनी है)
prompt = "A hyper-realistic portrait of Kavya, a gorgeous Indian woman with deep brown eyes and smooth, radiant skin. She is wearing a traditional red and gold saree with delicate embroidery, standing in soft natural sunlight. Her hair is neatly styled in a loose bun, with a few strands framing her face. The bokeh background enhances her beauty, capturing every fine detail of her facial features, ultra-HD, cinematic lighting, 8K resolution, DSLR photography, photorealistic, masterpiece."

# Image generate करें
image = pipeline(prompt=prompt, num_inference_steps=50, guidance_scale=7.5).images[0]

# Image दिखाएं
image.show()

# Image save करें
image.save("generated_image.png")


Token indices sequence length is longer than the specified maximum sequence length for this model (100 > 77). Running this sequence through the model will result in indexing errors
The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ['her facial features, ultra - hd, cinematic lighting, 8 k resolution, dslr photography, photorealistic, masterpiece.']
Token indices sequence length is longer than the specified maximum sequence length for this model (100 > 77). Running this sequence through the model will result in indexing errors
The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ['her facial features, ultra - hd, cinematic lighting, 8 k resolution, dslr photography, photorealistic, masterpiece.']


  0%|          | 0/50 [00:00<?, ?it/s]

In [ ]:
# Prompt दें (जो image generate करनी है)
prompt = "A stunning, hyper-realistic close-up of Kavya, a confident young woman with glowing skin, expressive brown eyes, and soft wavy hair. She is wearing a fitted black dress and subtle gold jewelry, standing on a balcony overlooking a city skyline at dusk. The golden hour lighting creates a warm, cinematic glow on her face, capturing fine skin textures, realistic shadows, and natural depth of field. Ultra-HD, professional photography, Vogue cover-style, 4K resolution, ultra-realistic, sharp focus"

# Image generate करें
image = pipeline(prompt=prompt, num_inference_steps=50, guidance_scale=7.5).images[0]

# Image दिखाएं
image.show()

# Image save करें
image.save("generated2_image.png")


The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ['and natural depth of field. ultra - hd, professional photography, vogue cover - style, 4 k resolution, ultra - realistic, sharp focus']
The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ['and natural depth of field. ultra - hd, professional photography, vogue cover - style, 4 k resolution, ultra - realistic, sharp focus']


  0%|          | 0/50 [00:00<?, ?it/s]

1. Confident City Look
"A stunning young woman with long, silky black hair and a radiant smile, sitting in a luxury car. She is wearing a sleek black outfit, her glowing skin highlighted by soft natural lighting. Her deep brown eyes reflect warmth and confidence. Ultra-realistic, 8K resolution, cinematic lighting, Vogue-style, DSLR shot, photorealistic, professional photography."

2. Golden Hour Elegance
"A gorgeous young woman with smooth, radiant skin, sitting in the golden hour sunlight. Her black hair flows effortlessly as she smiles warmly, looking directly into the camera. Natural light highlights her perfect facial structure. Ultra-HD, 8K, bokeh effect, soft focus, hyper-realistic photography."

3. Luxury Lifestyle
"A beautiful woman in a high-end luxury car, dressed in an elegant black dress. Her skin glows with natural beauty, and her deep brown eyes are full of life. Sunlight softly enters through the car window, creating a cinematic glow on her face. Ultra-HD, hyper-realistic, DSLR photography, soft bokeh, warm tones."

4. Natural Beauty Close-Up
"A hyper-realistic close-up portrait of a young woman with flawless, glowing skin. Her expressive eyes are full of warmth, and her deep black hair frames her delicate features. Soft light enhances the natural textures of her skin. Professional photography, ultra-HD, cinematic lighting, Vogue cover-style, 8K resolution."

5. Modern Minimalist Look
"A stylish young woman with sleek black hair, sitting confidently in a luxury car. She wears a simple black outfit with subtle gold jewelry. Her natural smile and glowing skin add charm to the scene. The background is softly blurred, keeping focus on her face. Ultra-HD, hyper-realistic, DSLR shot, cinematic soft lighting."

6. Urban Chic Aesthetic
"A beautiful woman with long flowing hair, sitting in a car with city lights softly glowing in the background. Her confident expression and radiant skin make her stand out. A perfect blend of elegance and modern style. Ultra-HD, cinematic, Vogue photoshoot, professional DSLR photography, 8K resolution."

7. Dreamy Portrait Photography
"A stunning woman sitting gracefully in a luxury car, her silky black hair illuminated by soft natural light. She has a gentle yet confident expression, with deep brown eyes gazing into the lens. The depth of field enhances the sharpness of her features, making it an ultra-realistic masterpiece. Professional portrait, cinematic lighting, 4K, hyper-realistic, dreamy aesthetic."

8. Elegant Evening Look
"A young woman in a black evening dress, sitting confidently in a luxury car. Her long, dark hair cascades over her shoulders, and her warm smile lights up the space. The interior lights create a soft, cinematic glow on her flawless skin. Ultra-HD, Vogue magazine style, cinematic photography, photorealistic, 8K resolution."

9. Soft and Radiant Beauty
"A natural beauty with radiant, glowing skin sitting inside a car. The soft daylight highlights her delicate features, making her face look flawless and smooth. She wears a casual yet stylish outfit, with a confident and relaxed posture. Hyper-realistic, ultra-HD, natural lighting, DSLR, cinematic look, 8K resolution."

10. Bold and Beautiful
"A confident young woman with a striking expression, her deep black hair framing her glowing face. The high-definition lighting captures every detail of her smooth skin and natural beauty. A soft bokeh background adds depth, making her the focus of this masterpiece. Ultra-HD, 8K resolution, cinematic photography, professional DSLR shot."

In [ ]:
# Prompt दें (जो image generate करनी है)
prompt = "A stunning young woman with long, silky black hair and a radiant smile, sitting in a luxury car. She is wearing a sleek black outfit, her glowing skin highlighted by soft natural lighting. Her deep brown eyes reflect warmth and confidence. Ultra-realistic, 8K resolution, cinematic lighting, Vogue-style, DSLR shot, photorealistic, professional photography."
negative_prompt = "blurry, distorted face, extra fingers, bad anatomy, over-sharpened, cartoonish, unnatural skin texture, unrealistic lighting, overexposed, unnatural facial expressions, low resolution, poor depth of field, artificial look"

# Image generate करें
image = pipeline(prompt=prompt,negative_prompt=negative_prompt, num_inference_steps=50, guidance_scale=7.5).images[0]

# Image दिखाएं
image.show()

# Image save करें
image.save("generated3_image.png")


  0%|          | 0/50 [00:00<?, ?it/s]

In [ ]:
# Prompt दें (जो image generate करनी है)
prompt = "A stunning young woman with long, silky black hair and a radiant smile, sitting in a luxury car. She is wearing a sleek black outfit, her glowing skin highlighted by soft natural lighting. Her deep brown eyes reflect warmth and confidence. Ultra-realistic, 8K resolution, cinematic lighting, Vogue-style, DSLR shot, photorealistic, professional photography.Kavya, a stunning young woman with long, silky black hair and deep brown eyes, sitting gracefully inside a luxury car. Her flawless skin glows under soft natural light. She wears a stylish black top, exuding confidence and sophistication. The background features a high-end car interior with premium leather seats. Ultra-HD, hyper-realistic, cinematic soft lighting, Vogue-style portrait, DSLR shot, photorealistic perfection, 8K resolution."
negative_prompt = "blurry, distorted face, extra fingers, bad anatomy, over-sharpened, cartoonish, unnatural skin texture, unrealistic lighting, overexposed, unnatural facial expressions, low resolution, poor depth of field, artificial look"

# Image generate करें
image = pipeline(prompt=prompt,negative_prompt=negative_prompt, num_inference_steps=50, guidance_scale=7.5).images[0]

# Image दिखाएं
image.show()

# Image save करें
image.save("kavya1_image.png")


The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ['stunning young woman with long, silky black hair and deep brown eyes, sitting gracefully inside a luxury car. her flawless skin glows under soft natural light. she wears a stylish black top, exuding confidence and sophistication. the background features a high - end car interior with premium leather seats. ultra - hd, hyper - realistic, cinematic soft lighting, vogue - style portrait, dslr shot, photorealistic perfection, 8 k resolution.']
The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ['stunning young woman with long, silky black hair and deep brown eyes, sitting gracefully inside a luxury car. her flawless skin glows under soft natural light. she wears a stylish black top, exuding confidence and sophistication. the background features a high - end car interior with premium leather seats. ultra - hd, hyper - realistic, cinemat

  0%|          | 0/50 [00:00<?, ?it/s]

In [ ]:
# Prompt दें (जो image generate करनी है)
prompt = "Kavya, a confident and successful businesswoman, sits in a modern office with a sleek laptop in front of her. She wears a well-fitted blazer with a stylish watch on her wrist. Her sharp, focused gaze and confident posture exude professionalism. The office has minimalistic decor with a large glass window showcasing a cityscape. Ultra-realistic, cinematic lighting, 8K UHD, photorealistic perfection, professional headshot."
negative_prompt = "blurry, distorted face, extra fingers, bad anatomy, over-sharpened, cartoonish, unnatural skin texture, unrealistic lighting, overexposed, unnatural facial expressions, low resolution, poor depth of field, artificial look"

# Image generate करें
image = pipeline(prompt=prompt,negative_prompt=negative_prompt, num_inference_steps=50, guidance_scale=7.5).images[0]

# Image दिखाएं
image.show()

# Image save करें
image.save("kavya2_image.png")


The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ['uhd, photorealistic perfection, professional headshot.']
The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ['uhd, photorealistic perfection, professional headshot.']


  0%|          | 0/50 [00:00<?, ?it/s]

In [ ]:
# Prompt दें (जो image generate करनी है)
prompt = "Kavya, a beautiful young woman with silky black hair, enjoying a warm cup of coffee in a cozy café. She sits near a window with golden-hour sunlight casting a soft glow on her flawless skin. Her gentle smile radiates warmth and positivity. The café is decorated with warm wooden tones and soft lighting. Ultra-HD, hyper-realistic, cinematic photography, depth of field effect, DSLR portrait, 8K resolution."
negative_prompt = "blurry, distorted face, extra fingers, bad anatomy, over-sharpened, cartoonish, unnatural skin texture, unrealistic lighting, overexposed, unnatural facial expressions, low resolution, poor depth of field, artificial look"

# Image generate करें
image = pipeline(prompt=prompt,negative_prompt=negative_prompt, num_inference_steps=50, guidance_scale=7.5).images[0]

# Image दिखाएं
image.show()

# Image save करें
image.save("kavya3_image.png")


The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ['of field effect, dslr portrait, 8 k resolution.']
The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ['of field effect, dslr portrait, 8 k resolution.']


  0%|          | 0/50 [00:00<?, ?it/s]

In [ ]:
# Prompt दें (जो image generate करनी है)
prompt = "Kavya, a young woman with long, flowing black hair, standing on a serene beach during sunset. The golden-orange hues of the sun reflect on her glowing skin. She wears a stylish white summer dress, slightly flowing in the wind. The ocean waves crash gently in the background. Her expression is calm and thoughtful. Ultra-realistic, cinematic photography, 8K UHD, soft golden lighting, dreamy aesthetic."
negative_prompt = "blurry, distorted face, extra fingers, bad anatomy, over-sharpened, cartoonish, unnatural skin texture, unrealistic lighting, overexposed, unnatural facial expressions, low resolution, poor depth of field, artificial look"

# Image generate करें
image = pipeline(prompt=prompt,negative_prompt=negative_prompt, num_inference_steps=50, guidance_scale=7.5).images[0]

# Image दिखाएं
image.show()

# Image save करें
image.save("kavya4_image.png")


The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ['hd, soft golden lighting, dreamy aesthetic.']
The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ['hd, soft golden lighting, dreamy aesthetic.']


  0%|          | 0/50 [00:00<?, ?it/s]

In [ ]:
# Prompt दें (जो image generate करनी है)
prompt = "A stylish young woman with long, dark hair walking confidently through a neon-lit city street at night. She wears a chic black leather jacket, exuding modern elegance and mystery. The background features vibrant neon lights reflecting on the wet pavement, creating a cyberpunk aesthetic. Ultra-HD, cinematic portrait, dramatic lighting, hyper-realistic depth of field, 8K resolution."
negative_prompt = "blurry, distorted face, extra fingers, bad anatomy, over-sharpened, cartoonish, unnatural skin texture, unrealistic lighting, overexposed, unnatural facial expressions, low resolution, poor depth of field, artificial look"

# Image generate करें
image = pipeline(prompt=prompt,negative_prompt=negative_prompt, num_inference_steps=50, guidance_scale=7.5).images[0]

# Image दिखाएं
image.show()

# Image save करें
image.save("kavya5_image.png")


The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ['.']
The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ['.']


  0%|          | 0/50 [00:00<?, ?it/s]

In [ ]:
# Prompt दें (जो image generate करनी है)
prompt = "A graceful young woman with dark, flowing hair sitting in a vintage-style library, deeply engrossed in a book. The warm golden light from an antique lamp softly illuminates her face. She wears classic, sophisticated attire, adding to her intellectual charm. The background is filled with beautifully arranged bookshelves, giving a cozy and nostalgic atmosphere. Ultra-realistic, cinematic photography, DSLR shot, soft-focus, 8K resolution."
negative_prompt = "blurry, distorted face, extra fingers, bad anatomy, over-sharpened, cartoonish, unnatural skin texture, unrealistic lighting, overexposed, unnatural facial expressions, low resolution, poor depth of field, artificial look"

# Image generate करें
image = pipeline(prompt=prompt,negative_prompt=negative_prompt, num_inference_steps=50, guidance_scale=7.5).images[0]

# Image दिखाएं
image.show()

# Image save करें
image.save("kavya6_image.png")


The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: [', dslr shot, soft - focus, 8 k resolution.']
The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: [', dslr shot, soft - focus, 8 k resolution.']


  0%|          | 0/50 [00:00<?, ?it/s]

In [ ]:
!huggingface-cli login



    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: write

In [ ]:
from huggingface_hub import create_repo, upload_folder

repo_id = "myspaces/lora_model"

# ✅ अगर Repo पहले से नहीं बना है, तो बनाएँ
create_repo(repo_id, exist_ok=True)

# ✅ अपने LoRA Model की Directory को Push करें
upload_folder(
    folder_path="/content/drive/MyDrive/woman_sdxl_LoRA",
    repo_id=repo_id,
    commit_message="Uploading fine-tuned LoRA model"
)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

events.out.tfevents.1742880546.ba94730803c5.8842.0:   0%|          | 0.00/104k [00:00<?, ?B/s]

events.out.tfevents.1742880546.ba94730803c5.8842.1:   0%|          | 0.00/3.37k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/myspaces/lora_model/commit/ef235b3601e0a11dc95bd12c1f506409852f39b1', commit_message='Uploading fine-tuned LoRA model', commit_description='', oid='ef235b3601e0a11dc95bd12c1f506409852f39b1', pr_url=None, repo_url=RepoUrl('https://huggingface.co/myspaces/lora_model', endpoint='https://huggingface.co', repo_type='model', repo_id='myspaces/lora_model'), pr_revision=None, pr_num=None)

In [ ]:
!ls /content/drive/MyDrive/woman_sdxl_LoRA

logs  pytorch_lora_weights.safetensors


In [ ]:
from huggingface_hub import upload_folder

repo_id = "myspaces/lora_model"  # अपना Hugging Face repo ID डालें

upload_folder(
    folder_path="/content/drive/MyDrive/woman_sdxl_LoRA",  # LoRA Model की Directory
    repo_id=repo_id,
    commit_message="Uploading all LoRA model files"
)


No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/myspaces/lora_model/commit/ef235b3601e0a11dc95bd12c1f506409852f39b1', commit_message='Uploading all LoRA model files', commit_description='', oid='ef235b3601e0a11dc95bd12c1f506409852f39b1', pr_url=None, repo_url=RepoUrl('https://huggingface.co/myspaces/lora_model', endpoint='https://huggingface.co', repo_type='model', repo_id='myspaces/lora_model'), pr_revision=None, pr_num=None)

In [ ]:
import json
import os

# JSON config define करें
adapter_config = {
    "peft_type": "LORA",
    "base_model_name_or_path": "stabilityai/stable-diffusion-xl-base-1.0",
    "inference_mode": True,
    "r": 4,
    "target_modules": ["to_q", "to_v"],
    "lora_alpha": 16,
    "lora_dropout": 0.05,
    "fan_in_fan_out": False,
    "bias": "none",
    "modules_to_save": None
}

# JSON file save करें
config_path = "/content/drive/MyDrive/woman_sdxl_LoRA/adapter_config.json"
os.makedirs(os.path.dirname(config_path), exist_ok=True)
with open(config_path, "w") as f:
    json.dump(adapter_config, f, indent=4)

print(f"✅ `{config_path}` created successfully!")


✅ `/content/drive/MyDrive/woman_sdxl_LoRA/adapter_config.json` created successfully!


In [ ]:
from huggingface_hub import upload_folder

repo_id = "myspaces/lora_model"  # अपना Hugging Face repo ID डालें

upload_folder(
    folder_path="/content/drive/MyDrive/woman_sdxl_LoRA",  # LoRA Model की Directory
    repo_id=repo_id,
    commit_message="Uploading all LoRA model files including adapter_config.json"
)


CommitInfo(commit_url='https://huggingface.co/myspaces/lora_model/commit/52226d407cc770f693cf4abb2d09a9e247b9b7cf', commit_message='Uploading all LoRA model files including adapter_config.json', commit_description='', oid='52226d407cc770f693cf4abb2d09a9e247b9b7cf', pr_url=None, repo_url=RepoUrl('https://huggingface.co/myspaces/lora_model', endpoint='https://huggingface.co', repo_type='model', repo_id='myspaces/lora_model'), pr_revision=None, pr_num=None)

In [ ]:
import torch
from diffusers import StableDiffusionXLPipeline, DPMSolverMultistepScheduler


In [ ]:
base_model = "stabilityai/stable-diffusion-xl-base-1.0"
pipe = StableDiffusionXLPipeline.from_pretrained(base_model, torch_dtype=torch.float16, variant="fp16")
pipe.to("cuda")  # GPU पर लोड करें
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)


model_index.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

Fetching 19 files:   0%|          | 0/19 [00:00<?, ?it/s]

model.fp16.safetensors:   0%|          | 0.00/1.39G [00:00<?, ?B/s]

model.fp16.safetensors:   0%|          | 0.00/246M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/565 [00:00<?, ?B/s]

...

diffusion_pytorch_model.fp16.safetensors:   0%|          | 0.00/5.14G [00:00<?, ?B/s]

diffusion_pytorch_model.fp16.safetensors:   0%|          | 0.00/167M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

diffusion_pytorch_model.fp16.safetensors:   0%|          | 0.00/167M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [ ]:
lora_model_path = "myspaces/lora_model"  # तुम्हारा Hugging Face LoRA model path
pipe.load_lora_weights(lora_model_path)


pytorch_lora_weights.safetensors:   0%|          | 0.00/23.4M [00:00<?, ?B/s]

In [ ]:
prompt = "A highly detailed and realistic portrait of a young Indian woman named Kavya, with long black wavy hair, deep brown expressive eyes, and a warm smile. She is wearing a traditional red saree with golden embroidery, standing in soft golden-hour lighting. The background is slightly blurred with warm, natural tones. The atmosphere is elegant, cinematic, and artistic. Ultra HD, 8K, hyper-realistic, professional photography."

negative_prompt = "low quality, blurry, distorted details, unnatural skin texture, overexposed, cartoonish, artifacts, unrealistic proportions."


In [ ]:
image = pipe(
    prompt=prompt,
    negative_prompt=negative_prompt,
    num_inference_steps=30,
    guidance_scale=7.5
).images[0]

image.show()
image.save('kavya.png')


Token indices sequence length is longer than the specified maximum sequence length for this model (86 > 77). Running this sequence through the model will result in indexing errors
The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ['k, hyper - realistic, professional photography.']
Token indices sequence length is longer than the specified maximum sequence length for this model (86 > 77). Running this sequence through the model will result in indexing errors
The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ['k, hyper - realistic, professional photography.']


  0%|          | 0/30 [00:00<?, ?it/s]